In [ ]:
import os
import numpy as np
import pandas as pd
from sklearn import tree
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
import utils.baseline_functions as base
import matplotlib.pyplot as plt

In [ ]:
# Import data
os.chdir('/Users/jingyuanhu/Desktop/Research/Interpretable Opioid')
SAMPLE = pd.read_csv('Data/PATIENT_TABLE.csv', delimiter = ",")

# Simple CART

In [ ]:
X = SAMPLE[['Gender', 'Age', 'Age_Gender', 'Num_presc',
            'Total_days_supply', 'Avg_MME', 'Concurrent_opioid', 'Concurrent_benzo',
            'Num_drug', 'Insurance_change', 'Codeine', 'Codeine_MME', 'Hydrocodone',
            'Hydrocodone_MME', 'Oxycodone', 'Oxycodone_MME', 'Morphine',
            'Morphine_MME', 'Hydromorphone', 'Hydromorphone_MME', 'Methadone',
            'Methadone_MME', 'Fentanyl', 'Fentanyl_MME', 'Oxymorphone',
            'Oxymorphone_MME', 'Medicaid', 'CommercialIns', 'Medicare',
            'CashCredit', 'MilitaryIns', 'WorkersComp', 'Other', 'IndianNation',
            'Change_Medicaid', 'Change_CommercialIns', 'Change_Medicare',
            'Change_CashCredit', 'Change_MilitaryIns', 'Change_WorkersComp',
            'Change_Other', 'Change_IndianNation']].values
y = SAMPLE['Long_term_user'].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state = 42, stratify = y)

# Cross-validation
# parameters = {'max_depth':[i for i in range(1,10)]}
# parameters = {'min_impurity_decrease':[0, 1e-5, 1e-4, 1e-3, 1e-2]}
# clf_decision_tree = GridSearchCV(tree.DecisionTreeClassifier(random_state = 42, class_weight='balanced'), parameters, cv=5)
# clf_decision_tree.fit(X_train, y_train)
# x_axis = [i for i in range(1,10)]
# plt.plot(x_axis, clf_decision_tree.cv_results_['mean_test_score'])

In [ ]:
# Train
clf_decision_tree = tree.DecisionTreeClassifier(max_depth = 5, random_state = 42, 
                                                class_weight='balanced', min_samples_leaf = 5,
                                                min_impurity_decrease = 1e-5)
clf_decision_tree.fit(X_train, y_train)

# Plot
fig, axe = plt.subplots(figsize=(40,10), dpi = 300)
tree.plot_tree(clf_decision_tree, feature_names = ['Gender', 'Age', 'Age_Gender', 'Num_presc',
                                                   'Total_days_supply', 'Avg_MME', 'Concurrent_opioid', 'Concurrent_benzo',
                                                   'Num_drug', 'Insurance_change', 'Codeine', 'Codeine_MME', 'Hydrocodone',
                                                   'Hydrocodone_MME', 'Oxycodone', 'Oxycodone_MME', 'Morphine',
                                                   'Morphine_MME', 'Hydromorphone', 'Hydromorphone_MME', 'Methadone',
                                                   'Methadone_MME', 'Fentanyl', 'Fentanyl_MME', 'Oxymorphone',
                                                   'Oxymorphone_MME', 'Medicaid', 'CommercialIns', 'Medicare',
                                                   'CashCredit', 'MilitaryIns', 'WorkersComp', 'Other', 'IndianNation',
                                                   'Change_Medicaid', 'Change_CommercialIns', 'Change_Medicare',
                                                   'Change_CashCredit', 'Change_MilitaryIns', 'Change_WorkersComp',
                                                   'Change_Other', 'Change_IndianNation'], \
               class_names = ['short','long'], filled = True, rounded = True, ax = axe, fontsize=10)

# fig.savefig('decisiontree_all.png')

In [ ]:
# AUC
y_pred_prob = clf_decision_tree.fit(X_train, y_train).predict_proba(X_test)[:,1]
fpr, tpr, _ = metrics.roc_curve(y_test, y_pred_prob)
p, r, _ = metrics.precision_recall_curve(y_test, y_pred_prob)
roc_auc = metrics.auc(fpr, tpr)
pr_auc = metrics.auc(r, p)

# PR
fig = plt.figure(figsize=(15,8))
ax1 = fig.add_subplot(1,2,1)
ax1.set_xlabel('Recall')
ax1.set_ylabel('Precision')
ax1.set_title('PR Curve')
# ROC
ax2 = fig.add_subplot(1,2,2)
ax2.set_xlabel('False Positive Rate')
ax2.set_ylabel('True Positive Rate')
ax2.set_title('ROC Curve')

ax1.plot(r, p, label="AUC =" + str(round(pr_auc, 4)))
ax2.plot(fpr, tpr, label="AUC =" + str(round(roc_auc, 4)))
ax1.legend(loc='upper right')    
ax2.legend(loc='lower right')
# fig.savefig('pr_roc_all.png')

In [ ]:
# higher F1 scores are generally better?
y_pred = clf_decision_tree.predict(X_test)
accuracy = clf_decision_tree.score(X_test, y_test)
recall = metrics.recall_score(y_test, y_pred)
precision = metrics.precision_score(y_test, y_pred)
f1_score = metrics.fbeta_score(y_test, y_pred, beta = 1)
f2_score = metrics.fbeta_score(y_test, y_pred, beta = 2)
brier = metrics.brier_score_loss(y_test, y_pred)
print(round(accuracy,4),round(recall,4),round(precision,4),round(f1_score,4),round(f2_score,4),round(brier,4))

### Without days supply

In [ ]:
X = SAMPLE[['Gender', 'Age', 'Age_Gender', 'Num_presc',
            'Avg_MME', 'Concurrent_opioid', 'Concurrent_benzo',
            'Num_drug', 'Insurance_change', 'Codeine', 'Codeine_MME', 'Hydrocodone',
            'Hydrocodone_MME', 'Oxycodone', 'Oxycodone_MME', 'Morphine',
            'Morphine_MME', 'Hydromorphone', 'Hydromorphone_MME', 'Methadone',
            'Methadone_MME', 'Fentanyl', 'Fentanyl_MME', 'Oxymorphone',
            'Oxymorphone_MME', 'Medicaid', 'CommercialIns', 'Medicare',
            'CashCredit', 'MilitaryIns', 'WorkersComp', 'Other', 'IndianNation',
            'Change_Medicaid', 'Change_CommercialIns', 'Change_Medicare',
            'Change_CashCredit', 'Change_MilitaryIns', 'Change_WorkersComp',
            'Change_Other', 'Change_IndianNation']].values
y = SAMPLE['Long_term_user'].values

# Train
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state = 42, stratify = y)
clf_decision_tree = tree.DecisionTreeClassifier(max_depth = 5, random_state = 42, 
                                                class_weight='balanced', min_samples_leaf = 5,
                                                min_impurity_decrease = 1e-5)
clf_decision_tree.fit(X_train, y_train)

# Decision tree
fig, axe = plt.subplots(figsize=(40,10), dpi = 300)
tree.plot_tree(clf_decision_tree, feature_names = ['Gender', 'Age', 'Age_Gender', 'Num_presc',
                                                   'Avg_MME', 'Concurrent_opioid', 'Concurrent_benzo',
                                                   'Num_drug', 'Insurance_change', 'Codeine', 'Codeine_MME', 'Hydrocodone',
                                                   'Hydrocodone_MME', 'Oxycodone', 'Oxycodone_MME', 'Morphine',
                                                   'Morphine_MME', 'Hydromorphone', 'Hydromorphone_MME', 'Methadone',
                                                   'Methadone_MME', 'Fentanyl', 'Fentanyl_MME', 'Oxymorphone',
                                                   'Oxymorphone_MME', 'Medicaid', 'CommercialIns', 'Medicare',
                                                   'CashCredit', 'MilitaryIns', 'WorkersComp', 'Other', 'IndianNation',
                                                   'Change_Medicaid', 'Change_CommercialIns', 'Change_Medicare',
                                                   'Change_CashCredit', 'Change_MilitaryIns', 'Change_WorkersComp',
                                                   'Change_Other', 'Change_IndianNation'], \
               class_names = ['short','long'], filled = True, rounded = True, ax = axe, fontsize=10)

# fig.savefig('decisiontree_partial.png')

# AUC
y_pred_prob = clf_decision_tree.fit(X_train, y_train).predict_proba(X_test)[:,1]
fpr, tpr, _ = metrics.roc_curve(y_test, y_pred_prob)
p, r, _ = metrics.precision_recall_curve(y_test, y_pred_prob)
roc_auc = metrics.auc(fpr, tpr)
pr_auc = metrics.auc(r, p)
fig = plt.figure(figsize=(15,8))
ax1 = fig.add_subplot(1,2,1)
ax1.set_xlabel('Recall')
ax1.set_ylabel('Precision')
ax1.set_title('PR Curve')
ax2 = fig.add_subplot(1,2,2)
ax2.set_xlabel('False Positive Rate')
ax2.set_ylabel('True Positive Rate')
ax2.set_title('ROC Curve')
ax1.plot(r, p, label="AUC =" + str(round(pr_auc, 4)))
ax2.plot(fpr, tpr, label="AUC =" + str(round(roc_auc, 4)))
ax1.legend(loc='upper right')    
ax2.legend(loc='lower right')
# fig.savefig('pr_roc_partial.png')

# Others
y_pred = clf_decision_tree.predict(X_test)
accuracy = clf_decision_tree.score(X_test, y_test)
recall = metrics.recall_score(y_test, y_pred)
precision = metrics.precision_score(y_test, y_pred)
f1_score = metrics.fbeta_score(y_test, y_pred, beta = 1)
f2_score = metrics.fbeta_score(y_test, y_pred, beta = 2)
brier = metrics.brier_score_loss(y_test, y_pred)
print(round(accuracy,4),round(recall,4),round(precision,4),round(f1_score,4),round(f2_score,4),round(brier,4))

# Baseline Models: 
- Decision Tree(CART), Explainable Boosting Machine (EBM)
- L1/L2 Logistic, Linear SVM, Random Forest, XGBoost

In [ ]:
import os
import csv
import numpy as np
import pandas as pd
from sklearn.utils import class_weight
import utils.baseline_functions as base

os.chdir('/Users/jingyuanhu/Desktop/Research/Interpretable_Opioid')
SAMPLE = pd.read_csv('Data/PATIENT_TABLE.csv', delimiter = ",")

In [ ]:
x = SAMPLE[['Gender', 'Age', 'Age_Gender', 'Num_presc',
            'Total_days_supply', 'Avg_MME', 'Concurrent_opioid', 'Concurrent_benzo',
            'Num_drug', 'Insurance_change', 'Codeine', 'Codeine_MME', 'Hydrocodone',
            'Hydrocodone_MME', 'Oxycodone', 'Oxycodone_MME', 'Morphine',
            'Morphine_MME', 'Hydromorphone', 'Hydromorphone_MME', 'Methadone',
            'Methadone_MME', 'Fentanyl', 'Fentanyl_MME', 'Oxymorphone',
            'Oxymorphone_MME', 'Medicaid', 'CommercialIns', 'Medicare',
            'CashCredit', 'MilitaryIns', 'WorkersComp', 'Other', 'IndianNation',
            'Change_Medicaid', 'Change_CommercialIns', 'Change_Medicare',
            'Change_CashCredit', 'Change_MilitaryIns', 'Change_WorkersComp',
            'Change_Other', 'Change_IndianNation']]
y = SAMPLE['Long_term_user'].values

In [ ]:
# Decision Tree: toy model to test different parameters
depth = [1,2,3,4]
min_samples = [5]
impurity = [0.001,0.01,0.1]
dt_summary = base.DecisionTree(X=x, Y=y, 
                               depth=depth, 
                               min_samples=min_samples, 
                               impurity=impurity,
                               seed=42)
dt_summary_balanced = base.DecisionTree(X=x, Y=y, 
                               depth=depth, 
                               min_samples=min_samples, 
                               impurity=impurity,
                               class_weight="balanced",
                               seed=42)

default = {"Accuracy": str(round(np.mean(dt_summary['holdout_test_accuracy']), 4)) + " (" + str(round(np.std(dt_summary['holdout_test_accuracy']), 4)) + ")",
           "Recall": str(round(np.mean(dt_summary['holdout_test_recall']), 4)) + " (" + str(round(np.std(dt_summary['holdout_test_recall']), 4)) + ")",
           "Precision": str(round(np.mean(dt_summary['holdout_test_precision']), 4)) + " (" + str(round(np.std(dt_summary['holdout_test_precision']), 4)) + ")",
           "ROC AUC": str(round(np.mean(dt_summary['holdout_test_roc_auc']), 4)) + " (" + str(round(np.std(dt_summary['holdout_test_roc_auc']), 4)) + ")",
           "PR AUC": str(round(np.mean(dt_summary['holdout_test_pr_auc']), 4)) + " (" + str(round(np.std(dt_summary['holdout_test_pr_auc']), 4)) + ")",
           "Brier": str(round(np.mean(dt_summary['holdout_test_brier']), 4)) + " (" + str(round(np.std(dt_summary['holdout_test_brier']), 4)) + ")",
           "F2": str(round(np.mean(dt_summary['holdout_test_f2']), 4)) + " (" + str(round(np.std(dt_summary['holdout_test_f2']), 4)) + ")"}

balanced = {"Accuracy": str(round(np.mean(dt_summary_balanced['holdout_test_accuracy']), 4)) + " (" + str(round(np.std(dt_summary_balanced['holdout_test_accuracy']), 4)) + ")",
           "Recall": str(round(np.mean(dt_summary_balanced['holdout_test_recall']), 4)) + " (" + str(round(np.std(dt_summary_balanced['holdout_test_recall']), 4)) + ")",
           "Precision": str(round(np.mean(dt_summary_balanced['holdout_test_precision']), 4)) + " (" + str(round(np.std(dt_summary_balanced['holdout_test_precision']), 4)) + ")",
           "ROC AUC": str(round(np.mean(dt_summary_balanced['holdout_test_roc_auc']), 4)) + " (" + str(round(np.std(dt_summary_balanced['holdout_test_roc_auc']), 4)) + ")",
           "PR AUC": str(round(np.mean(dt_summary_balanced['holdout_test_pr_auc']), 4)) + " (" + str(round(np.std(dt_summary_balanced['holdout_test_pr_auc']), 4)) + ")",
           "Brier": str(round(np.mean(dt_summary_balanced['holdout_test_brier']), 4)) + " (" + str(round(np.std(dt_summary_balanced['holdout_test_brier']), 4)) + ")",
           "F2": str(round(np.mean(dt_summary_balanced['holdout_test_f2']), 4)) + " (" + str(round(np.std(dt_summary_balanced['holdout_test_f2']), 4)) + ")"}

default = pd.DataFrame.from_dict(default, orient='index', columns=['Default'])
balanced = pd.DataFrame.from_dict(balanced, orient='index', columns=['Balanced'])
dt_results = pd.concat([default, balanced], axis=1)
dt_results.T

### Train model

In [ ]:
# Decision Tree
depth = [1,2,3,4,5]
min_samples = [5,10]
impurity = [0.001,0.01,0.1]
dt_summary = base.DecisionTree(X=x, Y=y, 
                               depth=depth, 
                               min_samples=min_samples, 
                               impurity=impurity,
                               seed=42)
dt_summary_balanced = base.DecisionTree(X=x, Y=y, 
                               depth=depth, 
                               min_samples=min_samples, 
                               impurity=impurity,
                               class_weight="balanced",
                               seed=42)

# EBM (parameters are changed)
# max_leaves = [1,2,3]
# min_samples = [1, 5, 10]
# ebm_summary = base.EBM(X=x, Y=y,
#                        max_leaves=max_leaves,
#                        min_samples=min_samples,
#                        seed=42)


# L2 logistic
c = np.linspace(1e-4,1,5).tolist()
logistic_summary = base.Logistic(X=x, Y=y, C=c, seed=42)
logistic_summary_balanced = base.Logistic(X=x, Y=y, C=c, class_weight="balanced", seed=42)

# L1 logistic
c = np.linspace(1e-4,1,5).tolist()
lasso_summary = base.Lasso(X=x, Y=y, C=c, seed=42)
lasso_summary_balanced = base.Lasso(X=x, Y=y, C=c, class_weight="balanced", seed=42)

# LinearSVM
c = np.linspace(1e-4,1,5).tolist()
svm_summary = base.LinearSVM(X=x, Y=y, C=c, seed=42)
svm_summary_balanced = base.LinearSVM(X=x, Y=y, C=c, class_weight="balanced", seed=42)

# Random Forest 
depth = [1,2,3,4,5]
n_estimators = [50,100,200]
impurity = [0.001,0.01]
rf_summary = base.RF(X=x, Y=y,
                     depth=depth,
                     estimators=n_estimators,
                     impurity=impurity,
                     seed=42)
rf_summary_balanced = base.RF(X=x, Y=y,
                              depth=depth,
                              estimators=n_estimators,
                              impurity=impurity,
                              class_weight="balanced",
                              seed=42)

# XGBoost
depth = [1,2,3,4,5]
n_estimators =  [50,100]
gamma = [5,10,15]
child_weight = [5,10,15]
xgb_summary = base.XGB(X=x, Y=y,
                       depth=depth,
                       estimators=n_estimators,
                       gamma=gamma,
                       child_weight=child_weight,
                       seed=42)
xgb_summary_balanced = base.XGB(X=x, Y=y,
                                depth=depth,
                                estimators=n_estimators,
                                gamma=gamma,
                                child_weight=child_weight,
                                class_weight="balanced",
                                seed=42)

### Export results

In [ ]:
# DT
default = {"Accuracy": str(round(np.mean(dt_summary['holdout_test_accuracy']), 4)) + " (" + str(round(np.std(dt_summary['holdout_test_accuracy']), 4)) + ")",
           "Recall": str(round(np.mean(dt_summary['holdout_test_recall']), 4)) + " (" + str(round(np.std(dt_summary['holdout_test_recall']), 4)) + ")",
           "Precision": str(round(np.mean(dt_summary['holdout_test_precision']), 4)) + " (" + str(round(np.std(dt_summary['holdout_test_precision']), 4)) + ")",
           "ROC AUC": str(round(np.mean(dt_summary['holdout_test_roc_auc']), 4)) + " (" + str(round(np.std(dt_summary['holdout_test_roc_auc']), 4)) + ")",
           "PR AUC": str(round(np.mean(dt_summary['holdout_test_pr_auc']), 4)) + " (" + str(round(np.std(dt_summary['holdout_test_pr_auc']), 4)) + ")",
           "Brier": str(round(np.mean(dt_summary['holdout_test_brier']), 4)) + " (" + str(round(np.std(dt_summary['holdout_test_brier']), 4)) + ")",
           "F2": str(round(np.mean(dt_summary['holdout_test_f2']), 4)) + " (" + str(round(np.std(dt_summary['holdout_test_f2']), 4)) + ")"}

balanced = {"Accuracy": str(round(np.mean(dt_summary_balanced['holdout_test_accuracy']), 4)) + " (" + str(round(np.std(dt_summary_balanced['holdout_test_accuracy']), 4)) + ")",
           "Recall": str(round(np.mean(dt_summary_balanced['holdout_test_recall']), 4)) + " (" + str(round(np.std(dt_summary_balanced['holdout_test_recall']), 4)) + ")",
           "Precision": str(round(np.mean(dt_summary_balanced['holdout_test_precision']), 4)) + " (" + str(round(np.std(dt_summary_balanced['holdout_test_precision']), 4)) + ")",
           "ROC AUC": str(round(np.mean(dt_summary_balanced['holdout_test_roc_auc']), 4)) + " (" + str(round(np.std(dt_summary_balanced['holdout_test_roc_auc']), 4)) + ")",
           "PR AUC": str(round(np.mean(dt_summary_balanced['holdout_test_pr_auc']), 4)) + " (" + str(round(np.std(dt_summary_balanced['holdout_test_pr_auc']), 4)) + ")",
           "Brier": str(round(np.mean(dt_summary_balanced['holdout_test_brier']), 4)) + " (" + str(round(np.std(dt_summary_balanced['holdout_test_brier']), 4)) + ")",
           "F2": str(round(np.mean(dt_summary_balanced['holdout_test_f2']), 4)) + " (" + str(round(np.std(dt_summary_balanced['holdout_test_f2']), 4)) + ")"}

default = pd.DataFrame.from_dict(default, orient='index', columns=['Decision Tree (Default)'])
balanced = pd.DataFrame.from_dict(balanced, orient='index', columns=['Decision Tree (Balanced)'])
dt_results = pd.concat([default, balanced], axis=1)

# L2
default = {"Accuracy": str(round(np.mean(logistic_summary['holdout_test_accuracy']), 4)) + " (" + str(round(np.std(logistic_summary['holdout_test_accuracy']), 4)) + ")",
           "Recall": str(round(np.mean(logistic_summary['holdout_test_recall']), 4)) + " (" + str(round(np.std(logistic_summary['holdout_test_recall']), 4)) + ")",
           "Precision": str(round(np.mean(logistic_summary['holdout_test_precision']), 4)) + " (" + str(round(np.std(logistic_summary['holdout_test_precision']), 4)) + ")",
           "ROC AUC": str(round(np.mean(logistic_summary['holdout_test_roc_auc']), 4)) + " (" + str(round(np.std(logistic_summary['holdout_test_roc_auc']), 4)) + ")",
           "PR AUC": str(round(np.mean(logistic_summary['holdout_test_pr_auc']), 4)) + " (" + str(round(np.std(logistic_summary['holdout_test_pr_auc']), 4)) + ")",
           "Brier": str(round(np.mean(logistic_summary['holdout_test_brier']), 4)) + " (" + str(round(np.std(logistic_summary['holdout_test_brier']), 4)) + ")",
           "F2": str(round(np.mean(logistic_summary['holdout_test_f2']), 4)) + " (" + str(round(np.std(logistic_summary['holdout_test_f2']), 4)) + ")"}

balanced = {"Accuracy": str(round(np.mean(logistic_summary_balanced['holdout_test_accuracy']), 4)) + " (" + str(round(np.std(logistic_summary_balanced['holdout_test_accuracy']), 4)) + ")",
           "Recall": str(round(np.mean(logistic_summary_balanced['holdout_test_recall']), 4)) + " (" + str(round(np.std(logistic_summary_balanced['holdout_test_recall']), 4)) + ")",
           "Precision": str(round(np.mean(logistic_summary_balanced['holdout_test_precision']), 4)) + " (" + str(round(np.std(logistic_summary_balanced['holdout_test_precision']), 4)) + ")",
           "ROC AUC": str(round(np.mean(logistic_summary_balanced['holdout_test_roc_auc']), 4)) + " (" + str(round(np.std(logistic_summary_balanced['holdout_test_roc_auc']), 4)) + ")",
           "PR AUC": str(round(np.mean(logistic_summary_balanced['holdout_test_pr_auc']), 4)) + " (" + str(round(np.std(logistic_summary_balanced['holdout_test_pr_auc']), 4)) + ")",
           "Brier": str(round(np.mean(logistic_summary_balanced['holdout_test_brier']), 4)) + " (" + str(round(np.std(logistic_summary_balanced['holdout_test_brier']), 4)) + ")",
           "F2": str(round(np.mean(logistic_summary_balanced['holdout_test_f2']), 4)) + " (" + str(round(np.std(logistic_summary_balanced['holdout_test_f2']), 4)) + ")"}

default = pd.DataFrame.from_dict(default, orient='index', columns=['Logistic (L2) (Default)'])
balanced = pd.DataFrame.from_dict(balanced, orient='index', columns=['Logistic (L2) (Balanced)'])
logistic_results = pd.concat([default, balanced], axis=1)


# L1
default = {"Accuracy": str(round(np.mean(lasso_summary['holdout_test_accuracy']), 4)) + " (" + str(round(np.std(lasso_summary['holdout_test_accuracy']), 4)) + ")",
           "Recall": str(round(np.mean(lasso_summary['holdout_test_recall']), 4)) + " (" + str(round(np.std(lasso_summary['holdout_test_recall']), 4)) + ")",
           "Precision": str(round(np.mean(lasso_summary['holdout_test_precision']), 4)) + " (" + str(round(np.std(lasso_summary['holdout_test_precision']), 4)) + ")",
           "ROC AUC": str(round(np.mean(lasso_summary['holdout_test_roc_auc']), 4)) + " (" + str(round(np.std(lasso_summary['holdout_test_roc_auc']), 4)) + ")",
           "PR AUC": str(round(np.mean(lasso_summary['holdout_test_pr_auc']), 4)) + " (" + str(round(np.std(lasso_summary['holdout_test_pr_auc']), 4)) + ")",
           "Brier": str(round(np.mean(lasso_summary['holdout_test_brier']), 4)) + " (" + str(round(np.std(lasso_summary['holdout_test_brier']), 4)) + ")",
           "F2": str(round(np.mean(lasso_summary['holdout_test_f2']), 4)) + " (" + str(round(np.std(lasso_summary['holdout_test_f2']), 4)) + ")"}

balanced = {"Accuracy": str(round(np.mean(lasso_summary_balanced['holdout_test_accuracy']), 4)) + " (" + str(round(np.std(lasso_summary_balanced['holdout_test_accuracy']), 4)) + ")",
           "Recall": str(round(np.mean(lasso_summary_balanced['holdout_test_recall']), 4)) + " (" + str(round(np.std(lasso_summary_balanced['holdout_test_recall']), 4)) + ")",
           "Precision": str(round(np.mean(lasso_summary_balanced['holdout_test_precision']), 4)) + " (" + str(round(np.std(lasso_summary_balanced['holdout_test_precision']), 4)) + ")",
           "ROC AUC": str(round(np.mean(lasso_summary_balanced['holdout_test_roc_auc']), 4)) + " (" + str(round(np.std(lasso_summary_balanced['holdout_test_roc_auc']), 4)) + ")",
           "PR AUC": str(round(np.mean(lasso_summary_balanced['holdout_test_pr_auc']), 4)) + " (" + str(round(np.std(lasso_summary_balanced['holdout_test_pr_auc']), 4)) + ")",
           "Brier": str(round(np.mean(lasso_summary_balanced['holdout_test_brier']), 4)) + " (" + str(round(np.std(lasso_summary_balanced['holdout_test_brier']), 4)) + ")",
           "F2": str(round(np.mean(lasso_summary_balanced['holdout_test_f2']), 4)) + " (" + str(round(np.std(lasso_summary_balanced['holdout_test_f2']), 4)) + ")"}

default = pd.DataFrame.from_dict(default, orient='index', columns=['Logistic (L1) (Default)'])
balanced = pd.DataFrame.from_dict(balanced, orient='index', columns=['Logistic (L1) (Balanced)'])
lasso_results = pd.concat([default, balanced], axis=1)


# SVM
default = {"Accuracy": str(round(np.mean(svm_summary['holdout_test_accuracy']), 4)) + " (" + str(round(np.std(svm_summary['holdout_test_accuracy']), 4)) + ")",
           "Recall": str(round(np.mean(svm_summary['holdout_test_recall']), 4)) + " (" + str(round(np.std(svm_summary['holdout_test_recall']), 4)) + ")",
           "Precision": str(round(np.mean(svm_summary['holdout_test_precision']), 4)) + " (" + str(round(np.std(svm_summary['holdout_test_precision']), 4)) + ")",
           "ROC AUC": str(round(np.mean(svm_summary['holdout_test_roc_auc']), 4)) + " (" + str(round(np.std(svm_summary['holdout_test_roc_auc']), 4)) + ")",
           "PR AUC": str(round(np.mean(svm_summary['holdout_test_pr_auc']), 4)) + " (" + str(round(np.std(svm_summary['holdout_test_pr_auc']), 4)) + ")",
           "Brier": str(round(np.mean(svm_summary['holdout_test_brier']), 4)) + " (" + str(round(np.std(svm_summary['holdout_test_brier']), 4)) + ")",
           "F2": str(round(np.mean(svm_summary['holdout_test_f2']), 4)) + " (" + str(round(np.std(svm_summary['holdout_test_f2']), 4)) + ")"}

balanced = {"Accuracy": str(round(np.mean(svm_summary_balanced['holdout_test_accuracy']), 4)) + " (" + str(round(np.std(svm_summary_balanced['holdout_test_accuracy']), 4)) + ")",
           "Recall": str(round(np.mean(svm_summary_balanced['holdout_test_recall']), 4)) + " (" + str(round(np.std(svm_summary_balanced['holdout_test_recall']), 4)) + ")",
           "Precision": str(round(np.mean(svm_summary_balanced['holdout_test_precision']), 4)) + " (" + str(round(np.std(svm_summary_balanced['holdout_test_precision']), 4)) + ")",
           "ROC AUC": str(round(np.mean(svm_summary_balanced['holdout_test_roc_auc']), 4)) + " (" + str(round(np.std(svm_summary_balanced['holdout_test_roc_auc']), 4)) + ")",
           "PR AUC": str(round(np.mean(svm_summary_balanced['holdout_test_pr_auc']), 4)) + " (" + str(round(np.std(svm_summary_balanced['holdout_test_pr_auc']), 4)) + ")",
           "Brier": str(round(np.mean(svm_summary_balanced['holdout_test_brier']), 4)) + " (" + str(round(np.std(svm_summary_balanced['holdout_test_brier']), 4)) + ")",
           "F2": str(round(np.mean(svm_summary_balanced['holdout_test_f2']), 4)) + " (" + str(round(np.std(svm_summary_balanced['holdout_test_f2']), 4)) + ")"}

default = pd.DataFrame.from_dict(default, orient='index', columns=['SVM (Default)'])
balanced = pd.DataFrame.from_dict(balanced, orient='index', columns=['SVM (Balanced)'])
svm_results = pd.concat([default, balanced], axis=1)


# RF
default = {"Accuracy": str(round(np.mean(rf_summary['holdout_test_accuracy']), 4)) + " (" + str(round(np.std(rf_summary['holdout_test_accuracy']), 4)) + ")",
           "Recall": str(round(np.mean(rf_summary['holdout_test_recall']), 4)) + " (" + str(round(np.std(rf_summary['holdout_test_recall']), 4)) + ")",
           "Precision": str(round(np.mean(rf_summary['holdout_test_precision']), 4)) + " (" + str(round(np.std(rf_summary['holdout_test_precision']), 4)) + ")",
           "ROC AUC": str(round(np.mean(rf_summary['holdout_test_roc_auc']), 4)) + " (" + str(round(np.std(rf_summary['holdout_test_roc_auc']), 4)) + ")",
           "PR AUC": str(round(np.mean(rf_summary['holdout_test_pr_auc']), 4)) + " (" + str(round(np.std(rf_summary['holdout_test_pr_auc']), 4)) + ")",
           "Brier": str(round(np.mean(rf_summary['holdout_test_brier']), 4)) + " (" + str(round(np.std(rf_summary['holdout_test_brier']), 4)) + ")",
           "F2": str(round(np.mean(rf_summary['holdout_test_f2']), 4)) + " (" + str(round(np.std(rf_summary['holdout_test_f2']), 4)) + ")"}

balanced = {"Accuracy": str(round(np.mean(rf_summary_balanced['holdout_test_accuracy']), 4)) + " (" + str(round(np.std(rf_summary_balanced['holdout_test_accuracy']), 4)) + ")",
           "Recall": str(round(np.mean(rf_summary_balanced['holdout_test_recall']), 4)) + " (" + str(round(np.std(rf_summary_balanced['holdout_test_recall']), 4)) + ")",
           "Precision": str(round(np.mean(rf_summary_balanced['holdout_test_precision']), 4)) + " (" + str(round(np.std(rf_summary_balanced['holdout_test_precision']), 4)) + ")",
           "ROC AUC": str(round(np.mean(rf_summary_balanced['holdout_test_roc_auc']), 4)) + " (" + str(round(np.std(rf_summary_balanced['holdout_test_roc_auc']), 4)) + ")",
           "PR AUC": str(round(np.mean(rf_summary_balanced['holdout_test_pr_auc']), 4)) + " (" + str(round(np.std(rf_summary_balanced['holdout_test_pr_auc']), 4)) + ")",
           "Brier": str(round(np.mean(rf_summary_balanced['holdout_test_brier']), 4)) + " (" + str(round(np.std(rf_summary_balanced['holdout_test_brier']), 4)) + ")",
           "F2": str(round(np.mean(rf_summary_balanced['holdout_test_f2']), 4)) + " (" + str(round(np.std(rf_summary_balanced['holdout_test_f2']), 4)) + ")"}

default = pd.DataFrame.from_dict(default, orient='index', columns=['Random Forest (Default)'])
balanced = pd.DataFrame.from_dict(balanced, orient='index', columns=['Random Forest (Balanced)'])
rf_results = pd.concat([default, balanced], axis=1)


# XGB
default = {"Accuracy": str(round(np.mean(xgb_summary['holdout_test_accuracy']), 4)) + " (" + str(round(np.std(xgb_summary['holdout_test_accuracy']), 4)) + ")",
           "Recall": str(round(np.mean(xgb_summary['holdout_test_recall']), 4)) + " (" + str(round(np.std(xgb_summary['holdout_test_recall']), 4)) + ")",
           "Precision": str(round(np.mean(xgb_summary['holdout_test_precision']), 4)) + " (" + str(round(np.std(xgb_summary['holdout_test_precision']), 4)) + ")",
           "ROC AUC": str(round(np.mean(xgb_summary['holdout_test_roc_auc']), 4)) + " (" + str(round(np.std(xgb_summary['holdout_test_roc_auc']), 4)) + ")",
           "PR AUC": str(round(np.mean(xgb_summary['holdout_test_pr_auc']), 4)) + " (" + str(round(np.std(xgb_summary['holdout_test_pr_auc']), 4)) + ")",
           "Brier": str(round(np.mean(xgb_summary['holdout_test_brier']), 4)) + " (" + str(round(np.std(xgb_summary['holdout_test_brier']), 4)) + ")",
           "F2": str(round(np.mean(xgb_summary['holdout_test_f2']), 4)) + " (" + str(round(np.std(xgb_summary['holdout_test_f2']), 4)) + ")"}

balanced = {"Accuracy": str(round(np.mean(xgb_summary_balanced['holdout_test_accuracy']), 4)) + " (" + str(round(np.std(xgb_summary_balanced['holdout_test_accuracy']), 4)) + ")",
           "Recall": str(round(np.mean(xgb_summary_balanced['holdout_test_recall']), 4)) + " (" + str(round(np.std(xgb_summary_balanced['holdout_test_recall']), 4)) + ")",
           "Precision": str(round(np.mean(xgb_summary_balanced['holdout_test_precision']), 4)) + " (" + str(round(np.std(xgb_summary_balanced['holdout_test_precision']), 4)) + ")",
           "ROC AUC": str(round(np.mean(xgb_summary_balanced['holdout_test_roc_auc']), 4)) + " (" + str(round(np.std(xgb_summary_balanced['holdout_test_roc_auc']), 4)) + ")",
           "PR AUC": str(round(np.mean(xgb_summary_balanced['holdout_test_pr_auc']), 4)) + " (" + str(round(np.std(xgb_summary_balanced['holdout_test_pr_auc']), 4)) + ")",
           "Brier": str(round(np.mean(xgb_summary_balanced['holdout_test_brier']), 4)) + " (" + str(round(np.std(xgb_summary_balanced['holdout_test_brier']), 4)) + ")",
           "F2": str(round(np.mean(xgb_summary_balanced['holdout_test_f2']), 4)) + " (" + str(round(np.std(xgb_summary_balanced['holdout_test_f2']), 4)) + ")"}

default = pd.DataFrame.from_dict(default, orient='index', columns=['XGB (Default)'])
balanced = pd.DataFrame.from_dict(balanced, orient='index', columns=['XGB (Balanced)'])
xgb_results = pd.concat([default, balanced], axis=1)


results = pd.concat([dt_results, logistic_results], axis=1)
results = pd.concat([results, lasso_results], axis=1)
results = pd.concat([results, svm_results], axis=1)
results = pd.concat([results, rf_results], axis=1)
results = pd.concat([results, xgb_results], axis=1)
results = results.T

In [ ]:
# Export results
path = './Results/baselines/'
results.to_csv(path + "results.csv")

# results_param = {"Decision Tree": dt_summary['best_param'],
#  "Explainable Boosting Machine": ebm_summary['best_param'],
#  "Logistic (L2)": logistic_summary['best_param'],
#  "Logistic (L1)": lasso_summary['best_param'],
#  "Linear SVM": svm_summary['best_param'],
#  "Random Forest": rf_summary['best_param'],
#  "XG Boost": xgb_summary['best_param']}
# results_param = pd.DataFrame.from_dict(results_param, orient='index')
# results_param

## Additive stumps

In [ ]:
import os
import csv
import random
import numpy as np
import pandas as pd
from sklearn.utils import class_weight
import utils.stumps as stumps
import time

os.chdir('/Users/jingyuanhu/Desktop/Research/Interpretable Opioid')
SAMPLE = pd.read_csv('Data/PATIENT_TABLE.csv', delimiter = ",")

x = SAMPLE[['Gender', 'Age', 'Age_Gender',
            'Num_presc', 'Total_days_supply','Concurrent_opioid', 'Concurrent_benzo',
            'Num_drug', 'Insurance_change', 'Codeine', 'Codeine_MME', 'Hydrocodone',
            'Hydrocodone_MME', 'Oxycodone', 'Oxycodone_MME', 'Morphine', 'Morphine_MME', 
            'Hydromorphone','Hydromorphone_MME', 'Methadone', 'Methadone_MME', 'Fentanyl',
            'Fentanyl_MME', 'Oxymorphone', 'Oxymorphone_MME', 'Medicaid',
            'CommercialIns', 'Medicare', 'CashCredit', 'MilitaryIns', 'WorkersComp',
            'Other', 'IndianNation', 'Medicaid_CommercialIns', 'Medicaid_Medicare',
            'Medicaid_CashCredit', 'Medicaid_MilitaryIns', 'Medicaid_WorkersComp',
            'Medicaid_Other', 'Medicaid_IndianNation', 'CommercialIns_Medicaid',
            'CommercialIns_Medicare', 'CommercialIns_CashCredit',
            'CommercialIns_MilitaryIns', 'CommercialIns_WorkersComp',
            'CommercialIns_Other', 'CommercialIns_IndianNation',
            'Medicare_Medicaid', 'Medicare_CommercialIns', 'Medicare_CashCredit',
            'Medicare_MilitaryIns', 'Medicare_WorkersComp', 'Medicare_Other',
            'Medicare_IndianNation', 'CashCredit_Medicaid',
            'CashCredit_CommercialIns', 'CashCredit_Medicare',
            'CashCredit_MilitaryIns', 'CashCredit_WorkersComp', 'CashCredit_Other',
            'CashCredit_IndianNation', 'MilitaryIns_Medicaid',
            'MilitaryIns_CommercialIns', 'MilitaryIns_Medicare',
            'MilitaryIns_CashCredit', 'MilitaryIns_WorkersComp',
            'MilitaryIns_Other', 'MilitaryIns_IndianNation', 'WorkersComp_Medicaid',
            'WorkersComp_CommercialIns', 'WorkersComp_Medicare',
            'WorkersComp_CashCredit', 'WorkersComp_MilitaryIns',
            'WorkersComp_Other', 'WorkersComp_IndianNation', 'Other_Medicaid',
            'Other_CommercialIns', 'Other_Medicare', 'Other_CashCredit',
            'Other_MilitaryIns', 'Other_WorkersComp', 'Other_IndianNation',
            'IndianNation_Medicaid', 'IndianNation_CommercialIns',
            'IndianNation_Medicare', 'IndianNation_CashCredit',
            'IndianNation_MilitaryIns', 'IndianNation_WorkersComp',
            'IndianNation_Other']]

# x = SAMPLE[['Gender', 'Age', 'Age_Gender', 'Num_presc',
#             'Total_days_supply', 'Avg_MME', 'Concurrent_opioid', 'Concurrent_benzo',
#             'Num_drug', 'Insurance_change', 'Codeine', 'Codeine_MME', 'Hydrocodone',
#             'Hydrocodone_MME', 'Oxycodone', 'Oxycodone_MME', 'Morphine',
#             'Morphine_MME', 'Hydromorphone', 'Hydromorphone_MME', 'Methadone',
#             'Methadone_MME', 'Fentanyl', 'Fentanyl_MME', 'Oxymorphone',
#             'Oxymorphone_MME', 'Medicaid', 'CommercialIns', 'Medicare',
#             'CashCredit', 'MilitaryIns', 'WorkersComp', 'Other', 'IndianNation',
#             'Change_Medicaid', 'Change_CommercialIns', 'Change_Medicare',
#             'Change_CashCredit', 'Change_MilitaryIns', 'Change_WorkersComp',
#             'Change_Other', 'Change_IndianNation']]

y = SAMPLE['Long_term_user'].values

In [ ]:
### COMPLETE STUMPS: cutoffs at all possible value

### Only need to be run once
# list of lists, each sublist contains all possible value of a feature

# cutoffs = []
# for column_name in x.columns:
#     # one cutoff for binary variables
#     if (len(x[column_name].unique()) == 2):
#         cutoffs.append([sorted(x[column_name].unique())[1]])
#     else:
#         cutoffs.append(sorted(x[column_name].unique()))

# new_data = stumps.create_stumps(x.values, x.columns, cutoffs)
# new_data.to_csv('Data/STUMPS.csv', header=True, index=False)

In [ ]:
# np.linspace(min(x['IndianNation_WorkersComp'].unique()), 
#             max(x['IndianNation_WorkersComp'].unique()), 
#             num = 3+1,
#             endpoint = False)[1:,].astype(int).tolist()

# sorted(x['IndianNation_MilitaryIns'].unique())
# len(x['IndianNation_MilitaryIns'].unique())

In [ ]:
### SMART STUMPS: instead of creating stumps for every possible value

### drop columns with only one value, no need to create stumps
columns_to_drop = []
for column_name in x.columns:
    if (len(x[column_name].unique()) == 1):
        columns_to_drop.append(column_name)
        
x = x.drop(columns=columns_to_drop)

### cutoffs equally selected
num_cutoff = 3
cutoffs = []

for column_name in x.columns:
    # one cutoff for binary variables
    if (len(x[column_name].unique()) == 2):
        cutoffs.append([sorted(x[column_name].unique())[1]])
    elif (len(x[column_name].unique()) == 3):
        cutoffs.append([sorted(x[column_name].unique())[2]])
    # at least four unique values
    else:
        cutoffs.append(np.linspace(min(x[column_name].unique()), 
                                   max(x[column_name].unique()), 
                                   num = num_cutoff+1,
                                   endpoint = False)[1:,].astype(int).tolist())

In [ ]:
# create stumps
new_data = stumps.create_stumps(x.values, x.columns, cutoffs)
new_data.to_csv('Data/SMART_STUMPS.csv', header=True, index=False)

In [ ]:
# additive stumps
start = time.time()
stump_summary = stumps.stump_cv(X = STUMPS, 
                                Y = y, 
                                columns=STUMPS.columns, 
                                c_grid={'C': np.linspace(1e-4,1e-1,10).tolist()},
                                seed = 42)
end = time.time()
print(str(round(end - start,1)) + ' seconds (default)')

start = time.time()
stump_summary_balanced = stumps.stump_cv(X = STUMPS, 
                                Y = y, 
                                columns=STUMPS.columns, 
                                c_grid={'C': np.linspace(1e-4,1e-1,10).tolist()},
                                class_weight = 'balanced',
                                seed = 42)
end = time.time()
print(str(round(end - start,1)) + ' seconds')

In [ ]:
default = {"Accuracy": str(round(np.mean(stump_summary['holdout_test_accuracy']), 4)) + " (" + str(round(np.std(stump_summary['holdout_test_accuracy']), 4)) + ")",
           "Recall": str(round(np.mean(stump_summary['holdout_test_recall']), 4)) + " (" + str(round(np.std(stump_summary['holdout_test_recall']), 4)) + ")",
           "Precision": str(round(np.mean(stump_summary['holdout_test_precision']), 4)) + " (" + str(round(np.std(stump_summary['holdout_test_precision']), 4)) + ")",
           "ROC AUC": str(round(np.mean(stump_summary['holdout_test_roc_auc']), 4)) + " (" + str(round(np.std(stump_summary['holdout_test_roc_auc']), 4)) + ")",
           "PR AUC": str(round(np.mean(stump_summary['holdout_test_pr_auc']), 4)) + " (" + str(round(np.std(stump_summary['holdout_test_pr_auc']), 4)) + ")",
           "Brier": str(round(np.mean(stump_summary['holdout_test_brier']), 4)) + " (" + str(round(np.std(stump_summary['holdout_test_brier']), 4)) + ")",
           "F2": str(round(np.mean(stump_summary['holdout_test_f2']), 4)) + " (" + str(round(np.std(stump_summary['holdout_test_f2']), 4)) + ")"}

balanced = {"Accuracy": str(round(np.mean(stump_summary_balanced['holdout_test_accuracy']), 4)) + " (" + str(round(np.std(stump_summary_balanced['holdout_test_accuracy']), 4)) + ")",
           "Recall": str(round(np.mean(stump_summary_balanced['holdout_test_recall']), 4)) + " (" + str(round(np.std(stump_summary_balanced['holdout_test_recall']), 4)) + ")",
           "Precision": str(round(np.mean(stump_summary_balanced['holdout_test_precision']), 4)) + " (" + str(round(np.std(stump_summary_balanced['holdout_test_precision']), 4)) + ")",
           "ROC AUC": str(round(np.mean(stump_summary_balanced['holdout_test_roc_auc']), 4)) + " (" + str(round(np.std(stump_summary_balanced['holdout_test_roc_auc']), 4)) + ")",
           "PR AUC": str(round(np.mean(stump_summary_balanced['holdout_test_pr_auc']), 4)) + " (" + str(round(np.std(stump_summary_balanced['holdout_test_pr_auc']), 4)) + ")",
           "Brier": str(round(np.mean(stump_summary_balanced['holdout_test_brier']), 4)) + " (" + str(round(np.std(stump_summary_balanced['holdout_test_brier']), 4)) + ")",
           "F2": str(round(np.mean(stump_summary_balanced['holdout_test_f2']), 4)) + " (" + str(round(np.std(stump_summary_balanced['holdout_test_f2']), 4)) + ")"}

default = pd.DataFrame.from_dict(default, orient='index', columns=['Additive Stumps (Default, Resample)'])
balanced = pd.DataFrame.from_dict(balanced, orient='index', columns=['Additive Stumps (Balanced, Resample)'])
stump_results = pd.concat([default, balanced], axis=1)
stump_results.T
# stump_results.to_csv('stump_results.csv', header=True, index=True)

In [ ]:
stump_summary

In [ ]:
stump_normal = pd.read_csv('stump_results.csv', delimiter = ",")

## RiskSlim
Because it is more time consuming, all the experiments are run on a sample dataset of size 400 (200 patient each class)

In [ ]:
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import recall_score, precision_score, roc_auc_score,\
average_precision_score, brier_score_loss, fbeta_score, accuracy_score

import pprint
import riskslim
import utils.RiskSLIM as slim
from riskslim.utils import print_model

os.chdir('/Users/jingyuanhu/Desktop/Research/Interpretable Opioid')

In [ ]:
### Preprocessing data

### Load stumps data
STUMPS = pd.read_csv('Data/SMART_STUMPS.csv', delimiter = ",")
SAMPLE_PATIENT = pd.read_csv('Data/SAMPLE_PATIENT_DATA.csv', delimiter = ",")

STUMPS['(Intercept)'] = 1
intercept = STUMPS.pop('(Intercept)')
STUMPS.insert(0, '(Intercept)', intercept)

# STUMPS.columns = ['(Intercept)', *STUMPS.columns[1:]]
x, y = STUMPS, SAMPLE_PATIENT.to_numpy()[:,[0]]
y[y==0]= -1

# STUMPS

In [ ]:
### Risk SLIM

# Training & testing
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.1, random_state = 25, stratify = y)

# problem parameters
max_coefficient = 5                                         # value of largest/smallest coefficient
max_L0_value = 5                                            # maximum model size
max_offset = 50                                             # maximum value of offset parameter (optional)
c0_value = 1e-6                                             # L0-penalty parameter such that c0_value > 0; larger values -> sparser models; we set to a small value (1e-6) so that we get a model with max_L0_value terms
w_pos = 1.00                                                # relative weight on examples with y = +1; w_neg = 1.00 (optional)
w_pos = len(y_train) / sum(y_train==1)[0]                   # balanced weight based on ratio

# load dataset
data = {
    'X': x_train.values,
    'Y': y_train,
    'variable_names': x_train.columns.tolist(),
    'outcome_name': 'Long_term_user',
    'sample_weights': np.repeat(1, len(y_train))
}

# coefficient set
coef_set = riskslim.CoefficientSet(variable_names = data['variable_names'], lb=-max_coefficient, ub=max_coefficient,
                                   sign=0)
coef_set.update_intercept_bounds(X = data['X'], y = data['Y'], max_offset = max_offset)

# create constraint dictionary
N, P = data['X'].shape
trivial_L0_max = P - np.sum(coef_set.C_0j == 0)
max_L0_value = min(max_L0_value, trivial_L0_max)

# run model
model_info, mip_info, lcpa_info = slim.risk_slim_constrain(data, 
                                                           max_coefficient=5, 
                                                           max_L0_value=5, 
                                                           c0_value=1e-6, 
                                                           max_offset=100, 
                                                           max_runtime=1000,
                                                           w_pos = w_pos)
print_model(model_info['solution'], data)
# pprint.pprint(model_info)

In [ ]:
### Risk SLIM (single) performance metric
y_test[y_test == -1] = 0

test_prob = slim.riskslim_prediction(x_test.values, np.array(x_test.columns.tolist()), model_info)
test_pred = (test_prob > 0.5)

accuracy = accuracy_score(y_test, test_pred)
recall = recall_score(y_test, test_pred)
roc_auc = roc_auc_score(y_test, test_prob)
pr_auc = average_precision_score(y_test, test_prob)

print(accuracy, recall, roc_auc, pr_auc)

---------------------------------------------------------------------------------------

In [ ]:
### Risk SLIM (nested CV)
### Only tune one hyperparameter for computational issues and to control variations
### has to equal the num_folds of inner_CV
max_coef_number = [1,2,3,4,5]

risk_summary = slim.risk_nested_cv_constrain(X=x, 
                                             Y=y,
                                             y_label='Long_term_user', 
                                             max_coef=5, 
                                             max_coef_number=max_coef_number,
                                             c=1e-6, 
                                             seed=42)

In [ ]:
max_coef_number = [1,2,3,4,5]

risk_summary_balanced = slim.risk_nested_cv_constrain(X=x,
                                                      Y=y,
                                                      y_label='Long_term_user', 
                                                      max_coef=5, 
                                                      max_coef_number=max_coef_number,
                                                      c=1e-6, 
                                                      class_weight='balanced',
                                                      seed=42)

In [ ]:
default = {"Accuracy": str(round(np.mean(risk_summary['holdout_test_accuracy']), 4)) + " (" + str(round(np.std(risk_summary['holdout_test_accuracy']), 4)) + ")",
           "Recall": str(round(np.mean(risk_summary['holdout_test_recall']), 4)) + " (" + str(round(np.std(risk_summary['holdout_test_recall']), 4)) + ")",
           "Precision": str(round(np.mean(risk_summary['holdout_test_precision']), 4)) + " (" + str(round(np.std(risk_summary['holdout_test_precision']), 4)) + ")",
           "ROC AUC": str(round(np.mean(risk_summary['holdout_test_roc_auc']), 4)) + " (" + str(round(np.std(risk_summary['holdout_test_roc_auc']), 4)) + ")",
           "PR AUC": str(round(np.mean(risk_summary['holdout_test_pr_auc']), 4)) + " (" + str(round(np.std(risk_summary['holdout_test_pr_auc']), 4)) + ")",
           "Brier": str(round(np.mean(risk_summary['holdout_test_brier']), 4)) + " (" + str(round(np.std(risk_summary['holdout_test_brier']), 4)) + ")",
           "F2": str(round(np.mean(risk_summary['holdout_test_f2']), 4)) + " (" + str(round(np.std(risk_summary['holdout_test_f2']), 4)) + ")"}

balanced = {"Accuracy": str(round(np.mean(risk_summary_balanced['holdout_test_accuracy']), 4)) + " (" + str(round(np.std(risk_summary_balanced['holdout_test_accuracy']), 4)) + ")",
           "Recall": str(round(np.mean(risk_summary_balanced['holdout_test_recall']), 4)) + " (" + str(round(np.std(risk_summary_balanced['holdout_test_recall']), 4)) + ")",
           "Precision": str(round(np.mean(risk_summary_balanced['holdout_test_precision']), 4)) + " (" + str(round(np.std(risk_summary_balanced['holdout_test_precision']), 4)) + ")",
           "ROC AUC": str(round(np.mean(risk_summary_balanced['holdout_test_roc_auc']), 4)) + " (" + str(round(np.std(risk_summary_balanced['holdout_test_roc_auc']), 4)) + ")",
           "PR AUC": str(round(np.mean(risk_summary_balanced['holdout_test_pr_auc']), 4)) + " (" + str(round(np.std(risk_summary_balanced['holdout_test_pr_auc']), 4)) + ")",
           "Brier": str(round(np.mean(risk_summary_balanced['holdout_test_brier']), 4)) + " (" + str(round(np.std(risk_summary_balanced['holdout_test_brier']), 4)) + ")",
           "F2": str(round(np.mean(risk_summary_balanced['holdout_test_f2']), 4)) + " (" + str(round(np.std(risk_summary_balanced['holdout_test_f2']), 4)) + ")"}


default = pd.DataFrame.from_dict(default, orient='index', columns=['Risk SLIM (Default)'])
balanced = pd.DataFrame.from_dict(balanced, orient='index', columns=['Risk SLIM (Balanced)'])
slim_results = pd.concat([default, balanced], axis=1)
slim_results.T